In [1]:
import tensorflow as tf
import numpy as np
import gym
import random
import matplotlib.pyplot as plt
import collections

In [2]:
ENV = "CartPole-v0"

CAPCITY = 2000

UPDAT_PRMT_PERIOD = 100
REPLAY_PERIOD = 3
BATCH_SIZE = 32

EPISODE = 10000
MAX_STEP = 300

ALPHA = 0.5
BETA_INIT = 0.1 

In [3]:
# proportional variant / used in class Memory
class SumTree():
    def __init__(self ,  ):
        self.capcity = CAPCITY
        self.alpha = ALPHA
    
        # struct of SumTree & memory for the transition
        self.tree = np.zeros( 2 * self.capcity - 1)
        self.data = [None] * self.capcity
        
        # pointer for the position
        self.pointer = 0
    
    # add new priority in leaf_node
    def add_leaf_node(self , p_alpha , transition):
        leaf_idex = self.pointer + self.capcity - 1
                
        self.data[self.pointer] = transition
        self.update_leaf_node(leaf_idex , p_alpha)
    
        self.pointer += 1
        if self.pointer >= self.capcity: # ！not self.capcity-1 
            self.pointer = 0
    
    # update leaf_node according leaf_idex
    def update_leaf_node(self , leaf_idex , p_alpha):
        change = p_alpha - self.tree[leaf_idex]
        self.tree[leaf_idex] = p_alpha 
        self._update_parent_node(change , leaf_idex )
            
    # update the value of sum p in parent node
    def _update_parent_node(self , change , child_idex ):
        parent_idex = (child_idex - 1) // 2
        
        self.tree[parent_idex] += change 
        
        if parent_idex != 0:
            self._update_parent_node(change , parent_idex)    
        
    # sampling to get leaf idex and transition
    def sampling(self , sample_idex):
        leaf_idex = self._retrieve(sample_idex)
        data_idex = leaf_idex - self.capcity + 1
        
        return [leaf_idex , self.tree[leaf_idex] , self.data[data_idex] ]
        
    # retrieve with O(log n)
    def _retrieve(self , sample_idex , node_idex = 0):
        left_child_idex = node_idex * 2 + 1
        right_child_idex = left_child_idex + 1
        
        if left_child_idex >= len(self.tree):  # ! must be  >= 
            return node_idex
        
        if self.tree[left_child_idex] == self.tree[right_child_idex]: 
            return self._retrieve(sample_idex , np.random.choice([left_child_idex , right_child_idex]))
        if self.tree[left_child_idex] > sample_idex:
            return self._retrieve(sample_idex , node_idex = left_child_idex)
        else:
            return self._retrieve(sample_idex - self.tree[left_child_idex] , node_idex = right_child_idex )
        
    # sum of p in root node
    def root_priority(self):
        return self.tree[0]

In [4]:
class Memory():
    def __init__(self , tree_epsilon = 0.01):
        self.epsilon = tree_epsilon
        self.p_init = 1. 
        self.beta = BETA_INIT
        self.beta_change_step = 0.001
        self.capcity = CAPCITY
        
        self.sum_tree = SumTree() 
        
    # store transition & priority before replay
    def store(self , transition):
        p_max = np.max(self.sum_tree.tree[-self.capcity:])
        if p_max == 0:
            p_max = self.p_init
        self.sum_tree.add_leaf_node(p_max , transition)
        
    # update SumTree
    def update(self , leaf_idex , td_error  ):
        p = np.abs(td_error) + self.epsilon
        p_alpha = np.power(p , ALPHA)
        
        for i in range(len(leaf_idex)):
            self.sum_tree.update_leaf_node(leaf_idex[i] , p_alpha[i] )
        
    # sample
    def sampling(self , batch_size ):
        batch_idex = []
        batch_transition = []
        batch_ISweight = []
        
        segment = self.sum_tree.root_priority() / batch_size
        
        for i in range(batch_size):
            low = segment * i
            high = segment * (i + 1)
            sample_idex = np.random.uniform(low , high)
            idex , p_alpha , transition  = self.sum_tree.sampling(sample_idex)
            prob = p_alpha / self.sum_tree.root_priority()
            batch_ISweight.append( np.power(self.capcity * prob , -self.beta) )
            batch_idex.append( idex )
            batch_transition.append( transition)

        i_maxiwi = np.power(self.capcity * np.min(self.sum_tree.tree[-self.capcity:]) / self.sum_tree.root_priority() , self.beta)

        batch_ISweight = np.array(batch_ISweight) * i_maxiwi 
        
        return batch_idex , batch_transition , batch_ISweight
        
    # change beta
    def change_beta(self):
        self.beta -= self.beta_change_step
        return np.min(1 , self.beta)

In [5]:
class AgentModel():
    def __init__(self , env , sess):
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
        self.step_size = 0.01
        self.gamma = 0.5
        self.sess = sess
        self.epsilon = 0.8
        self.build_net()
        self.sess.run(tf.global_variables_initializer())
        
    # net_frame
    def net_frame(self , clt_name , inputs ):
        weight_init = tf.random_normal_initializer()
        bias_init = tf.constant_initializer(0.1)
        
        layer1_units = 64
        layer2_units = 32
        
        with tf.variable_scope("layer1"):
            weights1 = tf.get_variable("weight", initializer = weight_init , collections = clt_name , 
                                      shape = [self.state_dim , layer1_units ])
            bias1 = tf.get_variable("bias" , initializer = bias_init , collections = clt_name , shape = [layer1_units] )
            wx_b1 = tf.matmul(inputs , weights1) + bias1
            h1 = tf.nn.relu(wx_b1)
            
        with tf.variable_scope("layer2"):
            weights2 = tf.get_variable("weight" , initializer = weight_init , collections = clt_name , 
                                        shape = [layer1_units , layer2_units])
            bias2 = tf.get_variable("bias" , initializer = weight_init , collections = clt_name , 
                                    shape = [layer2_units])
            wx_b2 = tf.matmul(h1 , weights2) + bias2
            h2 = tf.nn.relu(wx_b2)
            
        with tf.variable_scope("layer3"):
            weights3 = tf.get_variable("weight" , initializer = weight_init , collections = clt_name , 
                                        shape = [layer2_units , self.action_dim])
            bias3 = tf.get_variable("bias" , initializer = weight_init , collections = clt_name , 
                                    shape = [self.action_dim])
            q_out = tf.matmul(h2 , weights3) + bias3
        
        return q_out       
                        
    # build net
    def build_net(self):
        with tf.variable_scope("q_net"):
            clt_name_q = ["q_net_prmts" , tf.GraphKeys.GLOBAL_VARIABLES]
            self.inputs_q = tf.placeholder( dtype = tf.float32 , shape = [None , self.state_dim] , name = "q_inputs" )
            self.q_value = self.net_frame(clt_name_q , self.inputs_q)
        
        with tf.variable_scope("target_net"):
            clt_name_target = ["target_net_prmts" , tf.GraphKeys.GLOBAL_VARIABLES]
            self.inputs_target = tf.placeholder( dtype = tf.float32 , shape = [None , self.state_dim] , name = "q_target")
            self.q_target = self.net_frame(clt_name_target , self.inputs_target)
            
        with tf.variable_scope("loss"):
            self.target = tf.placeholder( dtype = tf.float32 , shape=[ None , self.action_dim ] , name="target" )
            self.ISweight = tf.placeholder( dtype = tf.float32 , shape = [ None , self.action_dim] , name = "wi")
            
            self.td_error = self.target - self.q_value
            self.loss = tf.reduce_sum(self.ISweight * tf.squared_difference(self.target , self.q_value ))  # ***
            
        with tf.variable_scope("train"):
            self.train_op = tf.train.RMSPropOptimizer(self.step_size).minimize(self.loss)
            
    # training
    def training(self , state , next_state , reward , action , batch_ISweight ):
        q_target , q_value = self.sess.run([self.q_target , self.q_value] , 
                                          feed_dict = {self.inputs_q : state , self.inputs_target : next_state })        
        
        target = reward + self.gamma * np.max(q_target)
        reform_target = q_value.copy()
        batch_idex = np.arange(BATCH_SIZE)
        reform_target[batch_idex , action] = target
        
        batch_ISweight = np.stack([batch_ISweight , batch_ISweight] , axis = -1 )
        
        _ , td_error , loss = self.sess.run([self.train_op , self.td_error , self.loss] ,
                                            feed_dict = {self.ISweight : batch_ISweight , 
                                                         self.inputs_q : state ,
                                                         self.target : reform_target })
        return td_error
    
    # update target_net prmt
    def update_target_prmts(self):
        q_value_prmts = tf.get_collection("q_net_prmts")
        q_target_prmts = tf.get_collection("target_net_prmts")
        self.sess.run( [tf.assign(v , t) for v , t in zip(q_value_prmts , q_target_prmts) ])
        print("updating target network prmts...")
        
    # chose action
    def chose_action(self , current_state):
        state = current_state[np.newaxis , :]
        q_value = self.sess.run(self.q_value , feed_dict={self.inputs_q : state})
        if np.random.random() > self.epsilon:
            action = np.argmax(q_value)
        else:
            action = np.random.randint(0 , self.action_dim)
        return action
    
    def greedy_action(self , current_state):
        current_state = current_state[np.newaxis , :]  
        q = self.sess.run(self.q_value , feed_dict={self.inputs_q : current_state} ) 
        action_greedy = np.argmax(q)
        return action_greedy
        

In [6]:
def train( env , agent , memory ):
    Transition = collections.namedtuple("Transition" , ["state" , "action" , "reward" , "next_state" , "done"])
    replay_iter = 0
    update_target_iter = 0
    for episode in range(EPISODE):
        state = env.reset()
        for step in range(MAX_STEP):
            action = agent.chose_action(state)
            next_state , reward , done , _ = env.step(action)
            replay_iter += 1
            
            memory.store(Transition(state , action , reward , next_state , done))
            
            if replay_iter > CAPCITY and replay_iter % REPLAY_PERIOD == 0:
                batch_idex , batch_transition , batch_ISweight = memory.sampling(BATCH_SIZE)
                batch_state ,  batch_action , batch_reward , batch_next_state , batch_done = map(np.array , zip(*batch_transition))
                td_error = agent.training(batch_state , batch_next_state , batch_reward , batch_action , batch_ISweight)

                batch_iter = np.arange(BATCH_SIZE)
                memory.update(batch_idex , td_error[batch_iter , batch_action])

            if replay_iter > CAPCITY and replay_iter % UPDAT_PRMT_PERIOD == 0:
                agent.update_target_prmts()
            
            if done:
                if episode % 50 == 0:
                    print("episode: %d ,  step: %d" %( episode , step ))
                break
            state = next_state
            
                
                

In [ ]:
if __name__ == "__main__":
    env = gym.make(ENV)
    memory = Memory()
    sess = tf.Session()
    agent = AgentModel(env , sess)
    train( env , agent , memory )